In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
file_path = "data_tropicana_year105_gdp.csv"
df_gdp = pd.read_csv(file_path, delimiter=",")  

print(df_gdp.head())

In [ ]:
file_path = "data_tropicana_year105.csv"
df = pd.read_csv(file_path, delimiter=",")  

print(df.head())

In [ ]:
df['avg_income_per_profession'] = df.groupby(['year', 'profession'])['income'].transform('mean')



In [ ]:

# Group by year and profession to get the count of each profession per year
profession_counts = df.groupby(['year', 'profession']).size().reset_index(name='profession_count')

# Get the total number of workers per year (sum of counts of all professions for each year)
total_workers_per_year = df.groupby('year').size().reset_index(name='total_workers')

# Merge the total workers per year into the profession_counts
profession_counts = profession_counts.merge(total_workers_per_year, on='year', how='left')

# Calculate the percentage of the workforce for each profession per year
profession_counts['percentage_of_workforce'] = (profession_counts['profession_count'] / profession_counts['total_workers']) * 100

# Merge this back into the original dataset
df = df.merge(profession_counts[['year', 'profession', 'percentage_of_workforce']], on=['year', 'profession'], how='left')

# Display the first few rows to see the result
print(df.head())

In [ ]:
import numpy as np
import pandas as pd

# Gender Distribution Calculation
gender_distribution = df.groupby(['year', 'sex']).size().unstack(fill_value=0)
gender_distribution['total'] = gender_distribution.sum(axis=1)
gender_distribution['female_percentage'] = gender_distribution['F'] / gender_distribution['total']

# Employment Rate Calculation
non_employed_professions = ['child', 'unemployed', 'homemaker', 'retired']
df['employed'] = ~df['profession'].isin(non_employed_professions)
employment_rate = df.groupby('year')['employed'].mean()  # Convert to percentage

# Gini Coefficient Calculation
def gini_coefficient(income_list):
    """Computes the Gini coefficient of a list of incomes."""
    incomes = np.sort(income_list)  # Sort incomes in ascending order
    n = len(incomes)
    if n == 0:
        return np.nan  # Avoid division by zero
    cumulative_income_sum = np.sum((np.arange(1, n + 1) * incomes))
    total_income_sum = np.sum(incomes)
    
    gini = (2 * cumulative_income_sum) / (n * total_income_sum) - (n + 1) / n
    return gini

gini = df.groupby('year')['income'].apply(gini_coefficient)







In [ ]:
#  Add column for average age per year
average_age_per_year = df.groupby('year')['age'].mean().reset_index(name='average_age')

# Merge average age per year into the original dataframe
df = df.merge(average_age_per_year, on='year', how='left')
